In [1]:
import os
import traceback
import logging
import os.path
import networkx as nx
import pandas as pd
import numpy as np
import pylab as pl
from matplotlib import collections  as mc

In [2]:
# simversion = '_v01'
simversion = '_plasmid_v01'
# simversion = '_plasmid_v02'
# simversion = '_plasmid_v03'
# simversion = '_plasmid_v04'
prepare_database_4_multicsar = 1
run_art = 1 # 1 True, 0 False
run_spades = 1
run_panta = 1
split_paralogs = 1
run_multicsar = 1
run_ragout = 0
run_pangraph = 1
# pangenome_data = '/data/hoan/amromics/data/ncbi/Kp100random/' # for pangraph, Ragout, multi-CSAR #v3
pangenome_data = '/data/hoan/amromics/data/ncbi/Ecoli50random/'
# pangenome_data = '/data/hoan/amromics/data/ncbi/Kptest/' # for pangraph, Ragout, multi-CSAR
scaffold_out_dir = '/data/hoan/amromics/genome-graph/scaffold_output/'

In [3]:
if prepare_database_4_multicsar==1:
    os.system('rm -r ' + pangenome_data + 'fna/')
    os.system('mkdir ' + pangenome_data + 'fna/')
    os.system('cp -r '+ pangenome_data + '*.fna.gz '+ pangenome_data + 'fna/')
    os.system('~/miniconda3/envs/panta/bin/gzip -d ' + pangenome_data + 'fna/*.fna.gz')

### Simulate reads using ART

In [4]:
# https://github.com/scchess/Art/blob/master/art_illumina_README

In [5]:
sim_dir = '/data/hoan/amromics/simulation/'
art_bin = '/data/hoan/amromics/simulation/art_bin/./art_illumina'

In [6]:
# ref_data = '/data/hoan/amromics/simulation/references/GCF_000240185.1_ASM24018v2_genomic.fasta'
# ref_data = '/data/hoan/amromics/simulation/references/GCF_000240185.1_ASM24018v2_genomic.fna'
# ref_data = '/data/hoan/amromics/data/ncbi/Ecolitest/GCF_002860105.1_ASM286010v1_genomic.fna'
ref_data = '/data/hoan/amromics/data/ncbi/Ecolitest/GCF_002899495.1_ASM289949v1_genomic.fna'
sim_name = ref_data.split("/")[-2] + '_' + ref_data.split("/")[-1][:-1][:13]
sim_output = '/data/hoan/amromics/simulation/art_output/sim_reads_' + sim_name + simversion[-4:]
# sim_output = '/data/hoan/amromics/simulation/art_output/paired_dat_test'

In [7]:
sim_output

'/data/hoan/amromics/simulation/art_output/sim_reads_Ecolitest_GCF_002899495_v01'

In [ ]:
if run_art: 
    if simversion[-3:]=='v01':
        os.system(art_bin+' -ss HS20 -sam -i '+ref_data+' -p -l 100 -f 70 -m 400 -s 60 -o '+sim_output) # _v01
    if simversion[-3:]=='v02':
        os.system(art_bin+' -ss HS20 -sam -i '+ref_data+' -p -l 100 -f 70 -m 400 -s 10 -o '+sim_output) # _v02
    if simversion[-3:]=='v03':
        os.system(art_bin+' -ss MSv3 -sam -i '+ref_data+' -p -l 250 -f 70 -m 400 -s 60 -o '+sim_output) # _v3
        # os.system(art_bin+' -ss HS25 -sam -i '+ref_data+' -p -l 150 -f 70 -m 400 -s 60 -o '+sim_output) # _v0
    if simversion[-3:]=='v04':
        os.system(art_bin+' -ss MSv3 -sam -i '+ref_data+' -p -l 250 -f 70 -m 400 -s 10 -o '+sim_output) # _v3


    ====================ART====================
             ART_Illumina (2008-2016)          
          Q Version 2.5.8 (June 7, 2016)       
     Contact: Weichun Huang <whduke@gmail.com> 
    -------------------------------------------



### Run SPADES assembly

In [ ]:
spades_output = '/data/hoan/amromics/simulation/art_output/' + 'spades_output_' + sim_name + simversion[-4:]
spades_output

In [ ]:
if run_spades:
    spades_bin ='~/miniconda3/envs/amromics/bin/spades.py'
    os.system(spades_bin+' --isolate -1 '+sim_output+'1.fq -2 '+sim_output+'2.fq -t 50 -o '+spades_output)

### Run panta

In [ ]:
simversionRef = '_'+ pangenome_data.split('/')[-2] +'_'+ sim_name + simversion
simversionRef

In [ ]:
# move spades ouput to panta dir
print('Pangenome data: ', pangenome_data)
if run_panta:
    os.system('cp '+spades_output+'/contigs.fasta ' + pangenome_data + 'g1.fna')
    os.system('rm ' + pangenome_data + 'g1.fna.gz')
    os.system('~/miniconda3/envs/panta/bin/gzip -k ' + pangenome_data + 'g1.fna')

In [ ]:
# Run panta (change output dir)
# https://askubuntu.com/questions/1252439/not-able-to-activate-conda-environment-through-os-system-command-in-python
conda_dir = 'source ~/miniconda3/etc/profile.d/conda.sh && conda activate panta && '
panta_bin =conda_dir+'python /data/hoan/amromics/panta/panta.py'
panta_output = '/data/hoan/amromics/panta/examples/test/pantaOut' + simversionRef
# panta_output = '/data/hoan/amromics/panta/examples/test/output_Kp100' 
if split_paralogs == 0:
    panta_output = panta_output + '_nosplit'
print('PANTA OUTPUT: ', panta_output)
if run_panta:
    if split_paralogs:
        cmd_panta = panta_bin + ' -p init -a ' +pangenome_data+ '*.fna.gz -o '+panta_output +' -as -s -i 85 -c 20 -e 0.01'
    else:
        print("No split ortholog")
        cmd_panta = panta_bin + ' -p init -a ' +pangenome_data+ '*.fna.gz -o '+panta_output +' -as'
    os.system(cmd_panta)

### Run pangraph

In [ ]:
try:
    %load_ext autoreload
    %autoreload 2
except Exception as e:
    logging.error(traceback.format_exc())
from pangraph import PanGraph

In [ ]:
# set parameters
data_dir = panta_output 
incomplete_sample_name = 'g1'
assem_dir = spades_output
fasta_gen = 'partial' # 'all', 'partial'

In [ ]:
pangraph_output_greedy = scaffold_out_dir + 'pangraph_greedy'+ simversionRef + '.fasta'
pangraph_output_opt = scaffold_out_dir + 'pangraph_opt'+ simversionRef + '.fasta'

In [ ]:
pangraph_output_opt

In [ ]:
if run_pangraph:
    # https://stackoverflow.com/questions/4990718/how-can-i-write-a-try-except-block-that-catches-all-exceptions
    pangraph = PanGraph(sample_info=None, gene_info=None, gene_position=None)
    try:
        maximum_matching = 'greedy'
        pangraph.run_pangraph_pipeline(data_dir, incomplete_sample_name, assem_dir, fasta_gen, pangraph_output_greedy, maximum_matching)
    except Exception as e:
        logging.error(traceback.format_exc())
        # Logs the error appropriately. 

In [ ]:
if run_pangraph:
    try:
        maximum_matching = 'opt'
        pangraph.RERUN_pangraph_pipeline(data_dir, incomplete_sample_name, assem_dir, fasta_gen, pangraph_output_opt, maximum_matching)
    except Exception as e:
        logging.error(traceback.format_exc())
        # Logs the error appropriately. 

In [ ]:
# try:
#     maximum_matching = 'greedy'
#     pangraph_output_greedy = spades_output + '/contigs_concat_'+ maximum_matching+ simversionRef + '.fasta'
#     pangraph.RERUN_pangraph_pipeline(data_dir, incomplete_sample_name, assem_dir, fasta_gen, pangraph_output_opt, maximum_matching)
# except Exception as e:
#     logging.error(traceback.format_exc())
#     # Logs the error appropriately. 

### Run scaffold methods

In [ ]:
### Multi-CSAR
conda_dir = 'source ~/miniconda3/etc/profile.d/conda.sh && conda activate py27 && '
csar_bin = conda_dir + '/data/hoan/amromics/assembly/Multi-CSAR/./multi-csar.php'

In [ ]:
multicsar_output = scaffold_out_dir + 'multicsar/multicsar' + simversionRef
if run_multicsar:
    os.system('rm ' + pangenome_data + 'fna/g1*')
    os.system(csar_bin +' -t '+pangenome_data+'g1.fna -r '+pangenome_data+'fna/ --nuc -o '+multicsar_output)
    # os.system(csar_bin +' -t '+pangenome_data+'g1.fna -r '+pangenome_data+'*.fna --nuc -o '+multicsar_output)

In [ ]:
multicsar_output

In [ ]:
### Ragout
ragout_bin = 'source ~/miniconda3/etc/profile.d/conda.sh && conda activate py27 && ragout '
ragout_output = scaffold_out_dir + 'multicsar/ragout' + simversionRef
if run_ragout:
    # write receipt file
    import glob
    ref_files_list = glob.glob(pangenome_data+'fna/*')
    n_files = len(ref_files_list)
    receipt_file_dir = '/data/hoan/amromics/assembly/Ragout/kp100'+simversionRef+'.rcp'
    f = open(receipt_file_dir, "w")
    f.write('.references = ')
    for idx in range(n_files-1):
        f.write('r' + str(idx) + ',')
    f.write('r'+str(n_files-1)+'\n')
    f.write(".target = mg1655\n\n")
    for idx in range(n_files):
        f.write('r'+str(idx)+'.fasta =' + ref_files_list[idx] + '\n')
    f.write('mg1655.fasta = ' + spades_output+'/contigs.fasta\n')
    f.close()

In [ ]:
if run_ragout:
    os.system(ragout_bin + receipt_file_dir + ' --outdir ' + ragout_output + ' --refine')
    # ragout kp100_v2.rcp --outdir output_Kp100p_v2/ --refine

In [ ]:
print(ragout_output)

### Quast 

In [ ]:
quast_output = '/data/hoan/amromics/genome-graph/scaffold_output/quastResults' + simversionRef
quast_output

In [ ]:
quast_bin = 'python /data/hoan/amromics/spades_quast/quast-5.2.0/quast.py '
spades_output_fasta = spades_output+'/contigs.fasta'
multicsar_output_fasta = multicsar_output +'/multi-csar.nuc.out.fna'
ragout_output_fasta = ragout_output + '/mg1655_scaffolds.fasta'
if 1:
    if os.path.isfile(ragout_output_fasta):
        os.system(quast_bin + pangraph_output_opt+' '+ pangraph_output_greedy+' '+ multicsar_output_fasta+' '+ragout_output_fasta+' '+spades_output_fasta+' '+
        '-l "Pangraph_OPT, Pangraph_Greedy, Multi-CSAR, Ragout, SPADES" '+ '-r '+ref_data+' -o '+ quast_output+' --silent --extensive-mis-size 5000 --local-mis-size 3000')
    else:
        os.system(quast_bin + pangraph_output_opt+' '+ pangraph_output_greedy+' '+ multicsar_output_fasta+' '+spades_output_fasta+' '+
        '-l "Pangraph_OPT, Pangraph_Greedy, Multi-CSAR, SPADES" '+ '-r '+ref_data+' -o '+ quast_output+' --silent --extensive-mis-size 5000 --local-mis-size 3000')

In [ ]:
# # pangraph.H["weight"][0:4]
# iteration = 0
# for u,v,a in pangraph.H.edges(data=True):
#     print(a)
#     iteration += 1
#     if iteration > 10:
#         break;

In [ ]:
# nx.write_gml(pangraph.assembly_graph, quast_output+'/assembly_graph.gml')

In [ ]:
pangraph.assembly_graph.number_of_edges()

In [ ]:
len(list(nx.weakly_connected_components(pangraph.assembly_graph)))

In [ ]:
# pangraph.weighted_CG.nodes()

In [ ]:
len(list(nx.weakly_connected_components(pangraph.H)))

In [ ]:
pangraph.H.number_of_nodes(), pangraph.H.number_of_edges()

## Analysis

In [ ]:
# minimap2 /data/hoan/amromics/data/ncbi/Ecolitest/GCF_002860105.1_ASM286010v1_genomic.fna /data/hoan/amromics/genome-graph/scaffold_output/pangraph_opt_Ecoli100random_Ecoli_plasmid_v01.fasta > contigs100vsref_mapping_v01a.paf
cmd_minimap = '~/miniconda3/envs/amromics/bin/minimap2 '+ref_data +' '+ spades_output +'/contigs.fasta' +' > scaffold_output/minimap_out/mapping.paf' 

In [ ]:
os.system(cmd_minimap)

In [ ]:
df = pd.read_csv("scaffold_output/minimap_out/mapping.paf", sep ="\t", header=None)

In [ ]:
final_df = df.sort_values(by=[7])

In [ ]:
final_df

In [ ]:
target_id = 0
gene_position = pd.read_csv(panta_output + '/gene_position.tsv', delimiter='\t', header=None)
gene_position.columns =['SampleID', 'ContigName', 'GeneSequence']
target_contigs = [gene_position.iloc[i, 1] for i in range(len(gene_position.index)) if gene_position.iloc[i, 0]==target_id]

In [ ]:
target_contigs_sorted = [final_df.iloc[i, 0]+'---'+str(final_df.iloc[i, 7])+'>'+str(final_df.iloc[i, 8]) for i in range(len(final_df.index)) if final_df.iloc[i, 0] in target_contigs]

In [ ]:
target_contigs_sorted

In [ ]:
# pangraph.adj_list0

In [ ]:
# mapping: dnadiff (mummer) 
# checkM: reads quality control
# test for several read simulator
# Don't use SPADES, shovil keep files
# Does the problem exist for other genomes

In [ ]:
# cmd_dnadiff = '~/miniconda3/envs/amromics/bin/dnadiff '+ref_data +' '+ spades_output +'/contigs.fasta'
# os.system(cmd_dnadiff)